In [435]:
BASE_DIR = '../'*3
DATASET_DIR = BASE_DIR+'code/util/dataset/dataset_dummy.ipynb'
EXTMODEL_DIR = BASE_DIR + 'code/CNN/cnn_ext_model/ext_model.ipynb'

In [606]:
%run {DATASET_DIR}
%run {EXTMODEL_DIR}

Using device: cpu
Using PyTorch version: 1.9.0


In [607]:
imagenet = DummyDataset('imagenet', 'select',[3,299,299],200)

200


In [608]:
CnnExtModel.set_macro('v3_preproc', 
          ['serial',
              ['conv',{'ksize':3,'stride':2,'chn':32, 'padding':'VALID'}],
              ['conv',{'ksize':3,'chn':32, 'padding':'VALID'}],
              ['conv',{'ksize':3,'chn':64, 'padding':'SAME'}],
              ['max',{'ksize':3, 'stride':2, 'padding':'VALID'}],
              ['conv',{'ksize':1,'chn':80,'padding':'VALID'}],
              ['conv',{'ksize':3,'chn':192,'padding':'VALID'}],
              ['max',{'ksize':3, 'stride':2,'padding':'VALID'}]
          
          ]
                     )

In [609]:
CnnExtModel.set_macro('v3_inception1',
        ['parallel',
            ['conv',{'ksize':1, 'chn':64}],
         
            ['serial',
                ['conv',{'ksize':1,'chn':48}],
                ['conv',{'ksize':5, 'chn':64}]],
         
            ['serial',
                ['conv',{'ksize':1,'chn':64}],
                 ['conv',{'ksize':3,'chn':96}],
                 ['conv',{'ksize':3,'chn':96}]],
         
            ['serial',
                ['avg',{'ksize':3,'stride':1}],
                ['conv',{'ksize':1,'chn':'#chn'}]]
        
        ]
                    
                     )

In [610]:
CnnExtModel.set_macro('v3_resize1',
            ['parallel',
                ['conv',{'ksize':3,'stride':2,'chn':384}],
                ['serial',
                 ['conv',{'ksize':1,'chn':64}],
                 ['conv',{'ksize':3,'chn':94}],
                 ['conv',{'ksize':3,'stride':2,'chn':96}]],
                ['max',{'ksize':3,'stride':2}]
            
            ]                     
                    
                     )

In [611]:
CnnExtModel.set_macro('v3_inception2',
    ['parallel',
        ['conv', {'ksize':1, 'chn':192}],
        ['serial',
            ['conv', {'ksize':[1,1], 'chn':'#chn'}],
            ['conv', {'ksize':[1,7], 'chn':'#chn'}],
            ['conv', {'ksize':[7,1], 'chn':192}]],
        ['serial',
            ['conv', {'ksize':[1,1], 'chn':'#chn'}],
            ['conv', {'ksize':[7,1], 'chn':'#chn'}],
            ['conv', {'ksize':[1,7], 'chn':'#chn'}],
            ['conv', {'ksize':[7,1], 'chn':'#chn'}],
            ['conv', {'ksize':[1,7], 'chn':192}]],
        ['serial',
            ['avg', {'ksize':3, 'stride':1}],
            ['conv', {'ksize':1, 'chn':192}]]])

In [612]:
CnnExtModel.set_macro('v3_resize2',
    ['parallel',
        ['serial',
            ['conv', {'ksize':1, 'chn':192}],
            ['conv', {'ksize':3, 'stride':2, 'chn':320}]],
        ['serial',
            ['conv', {'ksize':[1,1], 'chn':192}],
            ['conv', {'ksize':[1,7], 'chn':192}],
            ['conv', {'ksize':[7,1], 'chn':192}],
            ['conv', {'ksize':[3,3], 'stride':[2,2], 'chn':192}]],
        ['max', {'ksize':3, 'stride':2}]])

In [613]:
CnnExtModel.set_macro('v3_inception3',
    ['parallel',
        ['conv', {'ksize':1, 'chn':320}],
        ['serial',
            ['conv', {'ksize':[3,3], 'chn':384}],
            ['parallel',
                ['conv', {'ksize':[1,3], 'chn':384}],
                ['conv', {'ksize':[3,1], 'chn':384}]]],
        ['serial',
            ['conv', {'ksize':[1,1], 'chn':448}],
            ['conv', {'ksize':[3,3], 'chn':384}],
            ['parallel',
                ['conv', {'ksize':[1,3], 'chn':384}],
                ['conv', {'ksize':[3,1], 'chn':384}]]],
        ['serial',
            ['avg', {'ksize':3, 'stride':1}],
            ['conv', {'ksize':1, 'chn':192}]]])

In [614]:
CnnExtModel.set_macro('v3_postproc',
    ['serial',
        ['avg', {'stride':8}],
        ['dropout', {'keep_prob':0.7}]])

In [615]:
CnnExtModel.set_macro('inception_v3',
    ['serial',
        ['custom', {'name':'v3_preproc'}],
        ['custom', {'name':'v3_inception1', 'args':{'#chn':32}}],
        ['custom', {'name':'v3_inception1', 'args':{'#chn':64}}],
        ['custom', {'name':'v3_inception1', 'args':{'#chn':64}}],
        ['custom', {'name':'v3_resize1'}],
        ['custom', {'name':'v3_inception2', 'args':{'#chn':128}}],
        ['custom', {'name':'v3_inception2', 'args':{'#chn':160}}],
        ['custom', {'name':'v3_inception2', 'args':{'#chn':160}}],
        ['custom', {'name':'v3_inception2', 'args':{'#chn':192}}],
        ['custom', {'name':'v3_resize2'}],
        ['custom', {'name':'v3_inception3'}],
        ['custom', {'name':'v3_inception3'}],
        ['custom', {'name':'v3_postproc'}]])

In [616]:
inception_v3 = CnnExtModel('inception_v3', imagenet,
                   [['custom', {'name':'inception_v3'}]], dump_structure=True)

custom inception_v3
14
  serial
    custom v3_preproc
8
      serial
        1: conv, [3, 299, 299]=>[32, 148, 148]pm : 32x3x3x3+32 = 896
        2: conv, [32, 148, 148]=>[32, 146, 146]pm : 32x32x3x3+32 = 9248
        3: conv, [32, 146, 146]=>[64, 146, 146]pm : 64x32x3x3+64 = 18496
        4: max, [64, 146, 146]=>[64, 72, 72]
        5: conv, [64, 72, 72]=>[80, 72, 72]pm : 80x64x1x1+80 = 5200
        6: conv, [80, 72, 72]=>[192, 70, 70]pm : 192x80x3x3+192 = 138432
        7: max, [192, 70, 70]=>[192, 34, 34]
<class 'str'> v3_preproc namenamenamenamenamenamenamenamenamenamenamename
[Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=valid), Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=valid), Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same), MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=valid, dilation=1, ceil_mode=False), Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), padding=valid), Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), padding=

          89: conv, [1280, 8, 8]=>[384, 8, 8]pm : 384x1280x3x3+384 = 4424064
          parallel
            90: conv, [384, 8, 8]=>[384, 8, 8]pm : 384x384x1x3+384 = 442752
            91: conv, [384, 8, 8]=>[384, 8, 8]pm : 384x384x3x1+384 = 442752
        serial
          92: conv, [1280, 8, 8]=>[448, 8, 8]pm : 448x1280x1x1+448 = 573888
          93: conv, [448, 8, 8]=>[384, 8, 8]pm : 384x448x3x3+384 = 1548672
          parallel
            94: conv, [384, 8, 8]=>[384, 8, 8]pm : 384x384x1x3+384 = 442752
            95: conv, [384, 8, 8]=>[384, 8, 8]pm : 384x384x3x1+384 = 442752
        serial
          96: avg, [1280, 8, 8]=>[1280, 8, 8]
          97: conv, [1280, 8, 8]=>[192, 8, 8]pm : 192x1280x1x1+192 = 245952
<class 'str'> v3_inception3_11
1
Conv2d(1280, 320, kernel_size=(1, 1), stride=(1, 1), padding=same)
    custom v3_inception3
5
      parallel
        98: conv, [2048, 8, 8]=>[320, 8, 8]pm : 320x2048x1x1+320 = 655680
        serial
          99: conv, [2048, 8, 8]=>[384, 8, 8]pm

In [617]:
inception_v3.net

[Block(
   (v3_inception1_2): Sequential(
     (dict): ModuleDict(
       (v3_inception1_2_1): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), padding=same)
       (v3_inception1_2_2): Conv2d(192, 48, kernel_size=(1, 1), stride=(1, 1), padding=same)
       (v3_inception1_2_3): Conv2d(48, 64, kernel_size=(5, 5), stride=(1, 1), padding=same)
       (v3_inception1_2_4): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), padding=same)
       (v3_inception1_2_5): Conv2d(64, 96, kernel_size=(3, 3), stride=(1, 1), padding=same)
       (v3_inception1_2_6): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=same)
       (v3_inception1_2_7): AvgPool2d(kernel_size=(1, 1), stride=(1, 1), padding=0)
       (v3_inception1_2_8): Conv2d(192, 32, kernel_size=(1, 1), stride=(1, 1), padding=same)
     )
   )
 ),
 Block(
   (v3_inception1_3): Sequential(
     (dict): ModuleDict(
       (v3_inception1_3_1): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), padding=same)
       (v3_inception1_3_2

In [277]:
class Net(nn.Module):
    def __init__(self,name, module):
        super(Net, self).__inti__()
        self.module =  module
        self.name =name
        setattr(self, block_name, nn.Sequential())
        getattr(self, block_name.add(self.asdf()))
        
    def forward(self, x):
        cat_block = []
        for block in self.module:
            if not isinstance(block, list): cat_block.append(block(x))
            else:
                out = None

                for i in block:
                    if not isinstance(i, list): out = i(x)
                    else:
                        val1,val2 = i
                        out=torch.cat(val1(out),val2(out),dim=1)
                cat_block.append(out)
        out=torch.cat(cat_block,dim=1)
        return out
        
    def perallel(self)    
        for i in self.module:
            if 
    def get_layer(self)
        layer=(name, layer)
        return layer
    
for i in inception_v3.modules['v3_inception3_11']:
    print(i)
    print('\n')
    break

IndentationError: expected an indented block (Temp/ipykernel_10148/2155437296.py, line 13)

In [335]:
a={}
idx = 1
for i in inception_v3.modules['v3_inception3_11']:
    if not isinstance(i,list):
        a['v3_inception3_11'+str(idx)]=i
        idx += 1
    else:
        for j in i:
            if not isinstance(j, list):
                a['v3_inception3_11'+str(idx)]=j
                idx +=1
            else:
                k,m = j
                a['v3_inception3_11'+str(idx)]=k
                a['v3_inception3_11'+str(idx)]=mZ
# inception_v3.modules

In [336]:
a

{'v3_inception3_111': Conv2d(1280, 320, kernel_size=(1, 1), stride=(1, 1), padding=same),
 'v3_inception3_112': Conv2d(1280, 384, kernel_size=(3, 3), stride=(1, 1), padding=same),
 'v3_inception3_113': Conv2d(1280, 448, kernel_size=(1, 1), stride=(1, 1), padding=same),
 'v3_inception3_114': Conv2d(448, 384, kernel_size=(3, 3), stride=(1, 1), padding=same),
 'v3_inception3_115': AvgPool2d(kernel_size=(1, 1), stride=(1, 1), padding=0),
 'v3_inception3_116': Conv2d(1280, 192, kernel_size=(1, 1), stride=(1, 1), padding=same)}

In [343]:
class Net(nn.Module):
    def __init__(self, a):
        super(Net, self).__init__()
#         for i in a:
        self.layer=nn.Sequential(nn.ModuleDict(a))
#         self.layer.add_modules(*a)
        

In [344]:
b=Net(a)

In [345]:
b

Net(
  (layer): Sequential(
    (0): ModuleDict(
      (v3_inception3_111): Conv2d(1280, 320, kernel_size=(1, 1), stride=(1, 1), padding=same)
      (v3_inception3_112): Conv2d(1280, 384, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (v3_inception3_113): Conv2d(1280, 448, kernel_size=(1, 1), stride=(1, 1), padding=same)
      (v3_inception3_114): Conv2d(448, 384, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (v3_inception3_115): AvgPool2d(kernel_size=(1, 1), stride=(1, 1), padding=0)
      (v3_inception3_116): Conv2d(1280, 192, kernel_size=(1, 1), stride=(1, 1), padding=same)
    )
  )
)

In [305]:
cat_block = []
for block in inception_v3.modules['v3_inception3_11']:
    if not isinstance(block, list): cat_block.append(block(x))
    else:
        out = None
        
        for i in block:
            if not isinstance(i, list): out = i(x)
            else:
                val1,val2 = i
                out=torch.cat(val1(out),val2(out))
        cat_block.append(out)


Conv2d(1280, 384, kernel_size=(3, 3), stride=(1, 1), padding=same)
[Conv2d(384, 384, kernel_size=(1, 3), stride=(1, 1), padding=same), Conv2d(384, 384, kernel_size=(3, 1), stride=(1, 1), padding=same)]


In [299]:
cat_block

[Conv2d(1280, 320, kernel_size=(1, 1), stride=(1, 1), padding=same)]

In [123]:
block_name = 'v3_inception3_11'
pm_hidden=inception_v3.modules[2]
idx = 10
#'1. 최초 블럭하나가 와서 모듈이 4개면 총 4개로 리스트에 다긴다.
# 수정하자
class tmp(nn.Module):
    def __init__(self, pm_hidden, block_name, idx):
        super(tmp, self).__init__()
        self.tot = []
        self.block=[]
        setattr(self, block_name, nn.Sequential())

        #         self.add_module()

                
#                 setattr(self, node, module_node)
                
#         method=getattr(self, block_name)
#         method(*block)


        self.block = block
    def forward(self,x):
        
        pass
    def make_block(self):
        for name, module in pm_hidden.named_children():
        if len(list(module.named_children())) < 1 :
            self.block.append(name)
        else:
            chck=[isinstance(seq, nn.Sequential) for seq in module]

            if not True in chck:

                self.block.append(name)

            else:

                arr= [i.replace('.weight','') for i in module.state_dict() if i.split('.')[-1]=='weight']

                cat=[]
                tot=[]

                # 앞의nn과 비교
                for i in range(len(arr)-1):
                    val=arr[i].split('.')
                    val_1=arr[i+1].split('.')

                    if len(val)>1 and val[:-1] == val_1[:-1]:
                        if val[-1].split('_')[0] and val_1[-1].split('_')[0] == 'parallel' and val[-1]!=val_1[-1]:
                            cat.append(arr[i])
                            cat.append(arr[i+1])
                    else:
                        tot.append(arr[i])

                if len(cat)!=0 : tot.append(cat)
                self.tot = tot
                self.block.append(tot)
a=tmp(pm_hidden,block_name,idx)
#'2. 4개의 블럭을 named모듈로 검사하여 palalell이라는 단어가 있으면
#'3. 다시 리스트에 담에서 병렬처리를 하게 한다.

# inception_v3.modules[10].

In [122]:
a.block

['parallel_88',
 ['serial_89', ['serial_91.parallel_90', 'serial_91.parallel_91']],
 ['serial_92',
  'serial_93',
  ['serial_95.parallel_94', 'serial_95.parallel_95']],
 'parallel_97']

In [124]:
a.block

['parallel_16', 'parallel_18', 'parallel_21', 'parallel_23']

In [80]:
tmp=[i.replace('.weight','') for i in pm_hidden[3].state_dict() if i.split('.')[-1]=='weight']
# dictionary = {i : string for i,string in enumerate(tmp)}
# print(len(dictionary))
tmp  

['serial_97']

In [47]:
for i in range(len(dictionary)-1):
    if dictionary[i].split('.')[0]
    

SyntaxError: invalid syntax (Temp/ipykernel_10148/841384369.py, line 2)

In [39]:
print(tot)

['serial_92', 'serial_93', ['serial_95.parallel_94', 'serial_95.parallel_95']]


In [16]:
a.tot

[[]]

In [165]:
for i in inception_v3.modules[10][1]:
    print(i)
    break


[Conv2d(1280, 320, kernel_size=(1, 1), stride=(1, 1), padding=same)]


In [194]:
len(inception_v3.modules[10][1][1])

1

In [266]:
inception_v3.modules['v3_inception3_11']

[Conv2d(1280, 320, kernel_size=(1, 1), stride=(1, 1), padding=same),
 [Conv2d(1280, 384, kernel_size=(3, 3), stride=(1, 1), padding=same),
  [Conv2d(384, 384, kernel_size=(1, 3), stride=(1, 1), padding=same),
   Conv2d(384, 384, kernel_size=(3, 1), stride=(1, 1), padding=same)]],
 [Conv2d(1280, 448, kernel_size=(1, 1), stride=(1, 1), padding=same),
  Conv2d(448, 384, kernel_size=(3, 3), stride=(1, 1), padding=same),
  [Conv2d(384, 384, kernel_size=(1, 3), stride=(1, 1), padding=same),
   Conv2d(384, 384, kernel_size=(3, 1), stride=(1, 1), padding=same)]]]

TypeError: data type "list" not understood

In [201]:
a={}
a['1']= 2

In [202]:
a

{'1': 2}

In [55]:
a=[]
for i in range(4):
    a=[a]
    
print(a)

[[[[[]]]]]


In [49]:
# list(inception_v3.modules[10][1].named_children())
# list(inception_v3.modules[10][1].named_modules())
                [1,[2[4],3[4]]] <<<<<<
1. 리스트에 계속 담는다.
[1,2,3]
1->2->3->[4(3),5(3)]

[[4]]

cat[4(3(2(1(x)))).5(3(2(1(x))))]
양갈래로 나오면 -> 마지막값을 

[[1,2,3]]

list(inception_v3.modules[10][1].state_dict())

['serial_89.weight',
 'serial_89.bias',
 'serial_91.parallel_90.weight',
 'serial_91.parallel_90.bias',
 'serial_91.parallel_91.weight',
 'serial_91.parallel_91.bias']

In [37]:
arr=list(map(lambda x : x.replace('.weight','').replace('.bias',''), list(inception_v3.modules[10][2].state_dict())))
result=[]
for value in arr: 
    if value not in result: 
        result.append(value)
        
cat=[]
tot=[]
for i in range(len(result)-1):
    val=result[i].split('.')
    val_1=result[i+1].split('.')

    if len(val)>1 and val[:-1] == val_1[:-1]:
        if val[-1].split('_')[0] and val_1[-1].split('_')[0] == 'parallel':
            cat.append(result[i])
            cat.append(result[i+1])
    else:
        tot.append(result[i])
tot.append(cat)

In [38]:
print(tot)

['serial_92', 'serial_93', ['serial_95.parallel_94', 'serial_95.parallel_95']]


In [139]:
cat=[]
for i in range(len(result)-1):
    val=result[i].split('.')
    val_1=result[i+1].split('.')

    if len(val)>1 and val[:-1] == val_1[:-1]:
        if val[-1].split('_')[0] and val_1[-1].split('_')[0] == 'parallel':
            cat.append(result[i])
            cat.append(result[i+1])

In [140]:
cat

['serial_91.parallel_90', 'serial_91.parallel_91']

In [79]:
a=()
for i in result:


[['serial_89'], ['serial_91', 'parallel_90'], ['serial_91', 'parallel_91']]

In [23]:
block=[]

for name, module in inception_v3.modules[10].named_children():
    if name == 'parallel_91':
        
        
        for i,j in list(module.named_modules())[1:]:
            print(i, j)
            print(11)
#             break

serial_89 Conv2d(1280, 384, kernel_size=(3, 3), stride=(1, 1), padding=same)
11
serial_91 Sequential(
  (parallel_90): Conv2d(384, 384, kernel_size=(1, 3), stride=(1, 1), padding=same)
  (parallel_91): Conv2d(384, 384, kernel_size=(3, 1), stride=(1, 1), padding=same)
)
11
serial_91.parallel_90 Conv2d(384, 384, kernel_size=(1, 3), stride=(1, 1), padding=same)
11
serial_91.parallel_91 Conv2d(384, 384, kernel_size=(3, 1), stride=(1, 1), padding=same)
11


In [26]:
#'1. 최초 블럭하나가 와서 모듈이 4개면 총 4개로 리스트에 다긴다.
name = 'v3_inception3_11'

class tmp(nn.Module):
    def __init__(self, pm_hidden, block_name, idx):
        
        setattr(self, block_name, nn.Sequential())
                
        block=[]
#         self.add_module()
        for name, module in pm_hidden.named_children():
            if not isinstance(module, nn.Sequential):
                block.append((name, module))

            
            else:
                
                for node, module_node in list(module.named_modules())[1:]:
                    node = node.replace('.','_')
                    setattr(self, node, module_node)
                
        method=getattr(self, block_name)
        method(*block)



    def forward(self,x):
        
    
#'2. 4개의 블럭을 named모듈로 검사하여 palalell이라는 단어가 있으면
#'3. 다시 리스트에 담에서 병렬처리를 하게 한다.

# inception_v3.modules[10].

4

In [175]:
i.i=1

AttributeError: 'str' object has no attribute 'i'

In [167]:
inception_v3.modules[10].parallel_91.serial_89

Conv2d(1280, 384, kernel_size=(3, 3), stride=(1, 1), padding=same)

In [176]:
for name, i in inception_v3.modules[10].named_modules():
    print(name)
    break

In [174]:
inception_v3.modules[10]

Sequential(
  (parallel_88): Conv2d(1280, 320, kernel_size=(1, 1), stride=(1, 1), padding=same)
  (parallel_91): Sequential(
    (serial_89): Conv2d(1280, 384, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (serial_91): Sequential(
      (parallel_90): Conv2d(384, 384, kernel_size=(1, 3), stride=(1, 1), padding=same)
      (parallel_91): Conv2d(384, 384, kernel_size=(3, 1), stride=(1, 1), padding=same)
    )
  )
  (parallel_95): Sequential(
    (serial_92): Conv2d(1280, 448, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (serial_93): Conv2d(448, 384, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (serial_95): Sequential(
      (parallel_94): Conv2d(384, 384, kernel_size=(1, 3), stride=(1, 1), padding=same)
      (parallel_95): Conv2d(384, 384, kernel_size=(3, 1), stride=(1, 1), padding=same)
    )
  )
  (parallel_97): Sequential(
    (serial_96): AvgPool2d(kernel_size=(1, 1), stride=(1, 1), padding=0)
    (serial_97): Conv2d(1280, 192, kernel_size=(1, 1), stride=(1, 1

In [179]:
for name, module in inception_v3.modules[10].named_children():
    if name == 'parallel_91':
        
        
        
        for i,j in list(module.named_modules())[1:]:
            print(i)
#             break

        
#         modul
# #         module[0]
# #         print(list(module.state_dict()))
# #         a=list(map(lambda x : len(x.split('.')), list(module.state_dict())))
#         b=list(map(lambda x : x.split('.')[:-1], list(module.state_dict())))
#         tmp = []
#         for i in b:
#             chck = None
#             if chck == i :
#                 pass
#             else:
#                 getattr(modul)
            
#             print(i)
#         print(max(a))

#         print(list(module.modules()))

serial_89
serial_91
serial_91.parallel_90
serial_91.parallel_91


In [182]:
'serial_89'.replace('.','_')
# 'serial_91.parallel_90'.replace('.','_')

'serial_89'

In [141]:
for name, modules in list(inception_v3.modules[10]):
    print(name)

TypeError: cannot unpack non-iterable Conv2d object

In [399]:
for i in range(len(list(inception_v3.modules[10]))):
#     print(list(inception_v3.modules[10])[i])
    
    if isinstance(list(inception_v3.modules[10])[i],nn.Sequential):
        for j,f in list(inception_v3.modules[10])[i].named_modules():
            print(f)
    
    
    
#     for j in list(inception_v3.modules[10])[i]
    if i == 1:
        break
#     for j in [list(inception_v3.modules[10])[i]]:
#         print(j)

Sequential(
  (serial_0): Conv2d(1280, 384, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (serial_1): Sequential(
    (parallel_0): Conv2d(384, 384, kernel_size=(1, 3), stride=(1, 1), padding=same)
    (parallel_1): Conv2d(384, 384, kernel_size=(3, 1), stride=(1, 1), padding=same)
  )
)
Conv2d(1280, 384, kernel_size=(3, 3), stride=(1, 1), padding=same)
Sequential(
  (parallel_0): Conv2d(384, 384, kernel_size=(1, 3), stride=(1, 1), padding=same)
  (parallel_1): Conv2d(384, 384, kernel_size=(3, 1), stride=(1, 1), padding=same)
)
Conv2d(384, 384, kernel_size=(1, 3), stride=(1, 1), padding=same)
Conv2d(384, 384, kernel_size=(3, 1), stride=(1, 1), padding=same)


In [385]:
a=1
for i in list(inception_v3.modules[10]):
    if isinstance(i, nn.Sequential):
        for j in i :
            print(j)
            a+=1
            if a == 2 :
                break
            break

Conv2d(1280, 384, kernel_size=(3, 3), stride=(1, 1), padding=same)
Conv2d(1280, 448, kernel_size=(1, 1), stride=(1, 1), padding=same)
AvgPool2d(kernel_size=(1, 1), stride=(1, 1), padding=0)


In [241]:
for name in inception_v3.macro[10]:
    print(name)
#     break

parallel
{}
['conv', {'ksize': 1, 'chn': 320}]
['serial', {}, ['conv', {'ksize': [3, 3], 'chn': 384}], ['parallel', {}, ['conv', {'ksize': [1, 3], 'chn': 384}], ['conv', {'ksize': [3, 1], 'chn': 384}]]]
['serial', {}, ['conv', {'ksize': [1, 1], 'chn': 448}], ['conv', {'ksize': [3, 3], 'chn': 384}], ['parallel', {}, ['conv', {'ksize': [1, 3], 'chn': 384}], ['conv', {'ksize': [3, 1], 'chn': 384}]]]
['serial', {}, ['avg', {'ksize': 3, 'stride': 1}], ['conv', {'ksize': 1, 'chn': 192}]]


In [107]:
a=1
for i in inception_v3.modules[1].children():
    print(i,a)
    a+=1
#     break

Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), padding=same) 1
Sequential(
  (serial_0): Conv2d(192, 48, kernel_size=(1, 1), stride=(1, 1), padding=same)
  (serial_1): Conv2d(48, 64, kernel_size=(5, 5), stride=(1, 1), padding=same)
) 2
Sequential(
  (serial_0): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), padding=same)
  (serial_1): Conv2d(64, 96, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (serial_2): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=same)
) 3
Sequential(
  (serial_0): AvgPool2d(kernel_size=(1, 1), stride=(1, 1), padding=0)
  (serial_1): Conv2d(192, 32, kernel_size=(1, 1), stride=(1, 1), padding=same)
) 4


In [ ]:
class Parallel(nn.Module):
    def __init__(self, layer):
        super(Parallel, self).__init__()
        
        setattr(self, self.name, nn.Sequential(*layer))
        
        
    def forward(self,x):
        out = []
        for moduls in self.name.children():
            out.append(moduls(x))
#         out1 = self.layer1
            cat=torch.cat(out,dim=0)
        
        return cat

In [173]:
inception_v3.modules[10].

'Sequential'

In [171]:
for i in inception_v3.modules[10].children():
    print(i)

Conv2d(1280, 320, kernel_size=(1, 1), stride=(1, 1), padding=same)
Sequential(
  (serial_0): Conv2d(1280, 384, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (serial_1): Sequential(
    (parallel_0): Conv2d(384, 384, kernel_size=(1, 3), stride=(1, 1), padding=same)
    (parallel_1): Conv2d(384, 384, kernel_size=(3, 1), stride=(1, 1), padding=same)
  )
)
Sequential(
  (serial_0): Conv2d(1280, 448, kernel_size=(1, 1), stride=(1, 1), padding=same)
  (serial_1): Conv2d(448, 384, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (serial_2): Sequential(
    (parallel_0): Conv2d(384, 384, kernel_size=(1, 3), stride=(1, 1), padding=same)
    (parallel_1): Conv2d(384, 384, kernel_size=(3, 1), stride=(1, 1), padding=same)
  )
)
Sequential(
  (serial_0): AvgPool2d(kernel_size=(1, 1), stride=(1, 1), padding=0)
  (serial_1): Conv2d(1280, 192, kernel_size=(1, 1), stride=(1, 1), padding=same)
)


In [169]:
if isinstance(inception_v3.macro[10], list):
    for i in 
list(map(lambda x: print(x),inception_v3.macro[10]))

parallel
{}
['conv', {'ksize': 1, 'chn': 320}]
['serial', {}, ['conv', {'ksize': [3, 3], 'chn': 384}], ['parallel', {}, ['conv', {'ksize': [1, 3], 'chn': 384}], ['conv', {'ksize': [3, 1], 'chn': 384}]]]
['serial', {}, ['conv', {'ksize': [1, 1], 'chn': 448}], ['conv', {'ksize': [3, 3], 'chn': 384}], ['parallel', {}, ['conv', {'ksize': [1, 3], 'chn': 384}], ['conv', {'ksize': [3, 1], 'chn': 384}]]]
['serial', {}, ['avg', {'ksize': 3, 'stride': 1}], ['conv', {'ksize': 1, 'chn': 192}]]


[None, None, None, None, None, None]

In [188]:
{} in inception_v3.macro[10]

True

In [178]:
inception_v3.macro[10][4][0]

'serial'

In [158]:
for i in inception_v3.macro[10]:
    print(i)

parallel
{}
['conv', {'ksize': 1, 'chn': 320}]
['serial', {}, ['conv', {'ksize': [3, 3], 'chn': 384}], ['parallel', {}, ['conv', {'ksize': [1, 3], 'chn': 384}], ['conv', {'ksize': [3, 1], 'chn': 384}]]]
['serial', {}, ['conv', {'ksize': [1, 1], 'chn': 448}], ['conv', {'ksize': [3, 3], 'chn': 384}], ['parallel', {}, ['conv', {'ksize': [1, 3], 'chn': 384}], ['conv', {'ksize': [3, 1], 'chn': 384}]]]
['serial', {}, ['avg', {'ksize': 3, 'stride': 1}], ['conv', {'ksize': 1, 'chn': 192}]]


In [618]:
F_DATA_DIR = BASE_DIR+ 'code/util/dataset/flower_prepro.ipynb'

%run {F_DATA_DIR}

fd = FlowersDataset([96,96], [3,96,96])

Using device: cpu
Using PyTorch version: 1.9.0


In [619]:
CnnExtModel.set_macro('flower_preproc',
    ['serial',
        ['conv', {'ksize':3, 'stride':2, 'chn':6, 'actions':'#act'}]])

CnnExtModel.set_macro('flower_inception1',
    ['parallel',
        ['conv', {'ksize':1, 'chn':4, 'actions':'#act'}],
        ['conv', {'ksize':3, 'chn':6, 'actions':'#act'}],
        ['serial',
            ['conv', {'ksize':3, 'chn':6, 'actions':'#act'}],
            ['conv', {'ksize':3, 'chn':6, 'actions':'#act'}]],
        ['serial',
            ['avg', {'ksize':3, 'stride':1}],
            ['conv', {'ksize':1, 'chn':4, 'actions':'#act'}]]])

CnnExtModel.set_macro('flower_resize',
    ['parallel',
        ['conv', {'ksize':3, 'stride':2, 'chn':12, 'actions':'#act'}],
        ['serial',
            ['conv', {'ksize':3, 'chn':12, 'actions':'#act'}],
            ['conv', {'ksize':3, 'stride':2, 'chn':12, 'actions':'#act'}]],
        ['avg', {'ksize':3, 'stride':2}]])

CnnExtModel.set_macro('flower_inception2',
    ['parallel',
        ['conv', {'ksize':1, 'chn':8, 'actions':'#act'}],
        ['serial',
            ['conv', {'ksize':[3,3], 'chn':8, 'actions':'#act'}],
            ['parallel',
                ['conv', {'ksize':[1,3], 'chn':8, 'actions':'#act'}],
                ['conv', {'ksize':[3,1], 'chn':8, 'actions':'#act'}]]],
        ['serial',
            ['conv', {'ksize':[1,1], 'chn':8, 'actions':'#act'}],
            ['conv', {'ksize':[3,3], 'chn':8, 'actions':'#act'}],
            ['parallel',
                ['conv', {'ksize':[1,3], 'chn':8, 'actions':'#act'}],
                ['conv', {'ksize':[3,1], 'chn':8, 'actions':'#act'}]]],
        ['serial',
            ['avg', {'ksize':3, 'stride':1}],
            ['conv', {'ksize':1, 'chn':8, 'actions':'#act'}]]])

CnnExtModel.set_macro('flower_postproc',
    ['serial',
        ['avg', {'stride':6}],
        ['dropout', {'keep_prob':0.7}]])

In [620]:
CnnExtModel.set_macro('inception_flower',
    ['serial',
        ['custom', {'name':'flower_preproc', 'args':{'#act':'#act'}}],
        ['custom', {'name':'flower_inception1', 'args':{'#act':'#act'}}],
        ['custom', {'name':'flower_resize', 'args':{'#act':'#act'}}],
        ['custom', {'name':'flower_inception1', 'args':{'#act':'#act'}}],
        ['custom', {'name':'flower_resize', 'args':{'#act':'#act'}}],
        ['custom', {'name':'flower_inception2', 'args':{'#act':'#act'}}],
        ['custom', {'name':'flower_resize', 'args':{'#act':'#act'}}],
        ['custom', {'name':'flower_inception2', 'args':{'#act':'#act'}}],
        ['custom', {'name':'flower_postproc', 'args':{'#act':'#act'}}]])

In [621]:
conf_flower_LA = ['custom', {'name':'inception_flower', 'args':{'#act':'LA'}}]
model_flower_LA = CnnExtModel('model_flower_LA', fd, 
                         conf_flower_LA, dump_structure=True)

custom inception_flower
10
  serial
    custom flower_preproc
2
      serial
        1: conv, (3, 96, 96)=>[6, 48, 48]pm : 6x3x3x3+6 = 168
<class 'str'> flower_preproc namenamenamenamenamenamenamenamenamenamenamename
[Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1), padding=same)]
    custom flower_inception1
5
      parallel
        2: conv, [6, 48, 48]=>[4, 48, 48]pm : 4x6x1x1+4 = 28
        3: conv, [6, 48, 48]=>[6, 48, 48]pm : 6x6x3x3+6 = 330
        serial
          4: conv, [6, 48, 48]=>[6, 48, 48]pm : 6x6x3x3+6 = 330
          5: conv, [6, 48, 48]=>[6, 48, 48]pm : 6x6x3x3+6 = 330
        serial
          6: avg, [6, 48, 48]=>[6, 48, 48]
          7: conv, [6, 48, 48]=>[4, 48, 48]pm : 4x6x1x1+4 = 28
<class 'str'> flower_inception1_2
1
Conv2d(6, 4, kernel_size=(1, 1), stride=(1, 1), padding=same)
<class 'str'> flower_inception1_2
2
Conv2d(6, 6, kernel_size=(3, 3), stride=(1, 1), padding=same)
    custom flower_resize
4
      parallel
        8: conv, [20, 48, 48]=>[12, 24, 24]pm : 

In [622]:
model_flower_LA.exec_all(report=1)

Model model_flower_LA train started 




TypeError: list is not a Module subclass

In [623]:
model_flower_LA.layers

[[[Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1), padding=same)],
  [Conv2d(6, 4, kernel_size=(1, 1), stride=(1, 1), padding=same),
   Conv2d(6, 6, kernel_size=(3, 3), stride=(1, 1), padding=same),
   [Conv2d(6, 6, kernel_size=(3, 3), stride=(1, 1), padding=same),
    Conv2d(6, 6, kernel_size=(3, 3), stride=(1, 1), padding=same)],
   [AvgPool2d(kernel_size=(1, 1), stride=(1, 1), padding=0),
    Conv2d(6, 4, kernel_size=(1, 1), stride=(1, 1), padding=same)]],
  [Conv2d(20, 12, kernel_size=(3, 3), stride=(1, 1), padding=same),
   [Conv2d(20, 12, kernel_size=(3, 3), stride=(1, 1), padding=same),
    Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=same)],
   AvgPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0)],
  [Conv2d(44, 4, kernel_size=(1, 1), stride=(1, 1), padding=same),
   Conv2d(44, 6, kernel_size=(3, 3), stride=(1, 1), padding=same),
   [Conv2d(44, 6, kernel_size=(3, 3), stride=(1, 1), padding=same),
    Conv2d(6, 6, kernel_size=(3, 3), stride=(1, 1), padding=same)]